In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
import robustnessgym as rg
from tutorials.utils import load_train_data, score_predictions
import ujson as json
import jsonlines
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
from pathlib import Path
from collections import defaultdict
from bootleg.symbols.entity_symbols import EntitySymbols
from bootleg.symbols.type_symbols import TypeSymbols
from bootleg.symbols.kg_symbols import KGSymbols
import shutil
from IPython.core.display import display, HTML
from robustnessgym import ScoreSubpopulation, Dataset
from robustnessgym.core.identifier import Identifier

rg.Dataset.logdir = Path("/dfs/scratch0/lorr1/robustness_gym_cache")
rg.ScoreSubpopulation.logdir = Path("/dfs/scratch0/lorr1/robustness_gym_cache")
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_colwidth = 500
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5000)

[2021-03-09 14:04:48,662][INFO][root:41] :: Logging initialized.


In [49]:
def accuracy(df, crc_col="gold_qid", pred_col="pred_qid"):
    total = df.shape[0]
    correct = df[df[crc_col] == df[pred_col]].shape[0]
    return correct/total

def errors_by_type(df, type_sys, pred_col="pred_qid"):
    errors_type = defaultdict(int)
    df["correct"] = df["gold_qid"] == df[pred_col]
    for r in df.iterrows():
        row = r[1]
        if row.correct is True:
            continue
        for t in row[type_sys]:
            errors_type[t] += 1
    return errors_type

def add_columns(df, qid2cnt):
    if "cands" in df:
        df["num_cands"] = df["cands"].apply(lambda x: len(x))
        df["cand_names"] = df["cands"].apply(lambda x: [y[0] for y in x])
        df["cand_probs"] = df["cands"].apply(lambda x: [y[1] for y in x])
        del df["cands"]
    df["span"] = df["span"].apply(lambda x: tuple(x))
    df["in_cand"] = df.apply(lambda x: x["gold_title"] in x["cand_names"], axis=1)
    df["qid_cnt"] = df["gold_qid"].apply(lambda x: qid2cnt.get(x, 0))
    if "pred_qid" in df:
        df["pred_qid_cnt"] = df["pred_qid"].apply(lambda x: qid2cnt.get(x, 0))
        df = df[df["pred_qid"] != -1]
    # Get real aliases in case of contextual cands
    df["real_alias"] = df.apply(lambda x: " ".join(x["sentence"].split()[x["span"][0]:x["span"][1]]).lower(), axis=1)
    return df

In [7]:
data_dir = Path("/dfs/scratch0/lorr1/projects/bootleg-data/data/medmentions_0203")
data_subfolder = "ctx_30_exp_noNC"
emb_dir = data_dir / data_subfolder / "embs"
train_file = data_dir / data_subfolder / "train.jsonl"
test_file = data_dir / data_subfolder / "test.jsonl"
dev_file = data_dir / data_subfolder / "dev.jsonl"
print(f"Loading entity symbols")
es = EntitySymbols.load_from_cache(load_dir = data_dir / data_subfolder / "entity_db/entity_mappings", edit_mode=True)
a2q = es.get_alias2qids()
q2title = es.get_qid2title()
types_sym = TypeSymbols.load_from_cache(load_dir = data_dir / data_subfolder / "entity_db/type_mappings/umls")

Loading entity symbols


In [8]:
qid2cnt = defaultdict(int)
with jsonlines.open(train_file) as in_f:
    for line in in_f:
        for qid in line["qids"]:
            qid2cnt[qid] += 1

In [11]:
train_df = load_train_data(
    train_file, q2title, cands_map=a2q, type_symbols={"wiki": types_sym}, kg_symbols=None
)
dev_df = load_train_data(
    dev_file, q2title, cands_map=a2q, type_symbols={"wiki": types_sym}, kg_symbols=None
)
dev_df = add_columns(dev_df, qid2cnt)

100%|██████████| 8042/8042 [00:01<00:00, 6852.60it/s]


In [50]:
dev_pred_file = Path("/dfs/scratch1/lorr1/projects/bootleg/logs_medmentions_0203/cls_ctx/2021_02_25/08_36_54/a5f52b35/dev/last_model/bootleg_labels.jsonl")
dev_preds_df = score_predictions(orig_file=dev_file,
                 pred_file=dev_pred_file,
                 title_map=q2title,
                 cands_map=a2q,
                 type_symbols={"wiki": types_sym},
                 kg_symbols={})
dev_preds_df = add_columns(dev_preds_df, qid2cnt)

100%|██████████| 8042/8042 [00:01<00:00, 4681.23it/s]


In [51]:
dev_preds_df.columns
# If want to save
dev_preds_df["span"] = dev_preds_df["span"].apply(lambda x: list(x))
dev_preds_df.to_feather(dev_pred_file.parent / "dev_results.feather")
dev_preds_df["span"] = dev_preds_df["span"].apply(lambda x: tuple(x))

In [46]:
dev_preds_df.columns

Index(['sentence', 'sent_idx', 'aliases', 'span', 'slices', 'alias',
       'alias_idx', 'is_gold_label', 'gold_qid', 'pred_qid', 'gold_title',
       'pred_title', 'all_gold_qids', 'all_pred_qids', 'gold_label_aliases',
       'all_is_gold_labels', 'all_spans', 'wiki_gld', 'wiki_pred', 'gold',
       'spans', 'sent_idx_unq', 'doc_id', 'expandedAbbr', 'qids', 'num_cands',
       'cand_names', 'cand_probs', 'in_cand', 'qid_cnt', 'pred_qid_cnt',
       'real_alias'],
      dtype='object')

In [47]:
cols = ["sent_idx", "doc_id", "sentence", "alias", "real_alias", "in_cand", "qid_cnt", "pred_qid_cnt",
        "span", "gold_qid", "gold_title", "pred_qid", "pred_title",
        "wiki_gld", "wiki_pred", "all_spans", "all_gold_qids", "all_pred_qids", "aliases", "cand_names", "cand_probs"]

In [52]:
dev_preds_df[(dev_preds_df["pred_qid"] != dev_preds_df["gold_qid"]) & (dev_preds_df["doc_id"] == "27463942")][cols]

,sent_idx,doc_id,sentence,alias,real_alias,in_cand,qid_cnt,pred_qid_cnt,span,gold_qid,gold_title,pred_qid,pred_title,wiki_gld,wiki_pred,all_spans,all_gold_qids,all_pred_qids,aliases,cand_names,cand_probs
35643,6563,27463942,"Apixaban 5 mg Twice Daily and Clinical Outcomes in Patients With Atrial Fibrillation and Advanced Age , Low Body Weight , or High Creatinine : A Secondary Analysis of a Randomized Clinical Trial In the Apixaban for Reduction of Stroke and Other Thromboembolic Complications in Atrial Fibrillation ( ARISTOTLE ) trial , the standard dose of apixaban was 5 mg twice daily ; patients with at least 2 dose - reduction criteria-80 years or older , weight 60 kg or less , and creatinine level 1.5 mg/dL...",dev_35643,thromboembolic complications,True,5,2,"(42, 44)",C0040038,Thromboembolism,C0019087,Hemorrhagic Disorders,[Biologic Function],[Biologic Function],"[[0, 1], [11, 13], [23, 24], [30, 33], [35, 36], [39, 40], [42, 44], [45, 47], [48, 49], [50, 51], [56, 57], [83, 85], [93, 94]]","[C1831808, C0004238, C0010294, C0206034, C1831808, C0038454, C0040038, C0004238, C0008976, C0008976, C1831808, C0428279, C1831808]","[C1831808, C0004238, C0010294, C0206034, C1831808, C0038454, C0019087, C0004238, C0086418, C0008976, C1831808, C1278055, C1831808]","[dev_35637, dev_35638, dev_35639, dev_35640, dev_35641, dev_35642, dev_35643, dev_35644, dev_35645, dev_35646, dev_35647, dev_35648, dev_35649]","[Pathologic Processes, Postoperative Complications, Thromboembolism, Complication, Sequela of disorder, Iatrogenic Disease, Thrombosis, Chronobiology Disorders, Disease, Cardiac complication, Iatrogenesis, Sign or Symptom, Myelofibrosis, Thrombocytopenia, Hepatic Complication, Hemorrhagic Disorders, Pathognomic facies, Accidents and Adverse Effects, Risk of thromboembolic stroke, Thromboembolic stroke, Disease Response, Thromboembolism in childbirth, Hemorrhagic complication of pregnancy, Em...","[0.0006402123, 0.1485366523, 0.0604611114, 0.0159936901, 8.4597e-05, 0.0001463747, 0.0037168411, 6.04345e-05, 0.0026274738, 0.0145825297, 0.0035082949, 0.0023917581, 0.0009953292, 0.0012961165, 0.0133302361, 0.4777791798, 0.0009072023, 0.0034368902, 0.0101537965, 0.0320287496, 5.0167e-05, 0.005043535, 0.1932601482, 0.0039229416, 2.48669e-05, 0.0009964006, 0.00182055, 0.0002104343, 0.0001879608, 0.0018054832]"
35645,6563,27463942,"Apixaban 5 mg Twice Daily and Clinical Outcomes in Patients With Atrial Fibrillation and Advanced Age , Low Body Weight , or High Creatinine : A Secondary Analysis of a Randomized Clinical Trial In the Apixaban for Reduction of Stroke and Other Thromboembolic Complications in Atrial Fibrillation ( ARISTOTLE ) trial , the standard dose of apixaban was 5 mg twice daily ; patients with at least 2 dose - reduction criteria-80 years or older , weight 60 kg or less , and creatinine level 1.5 mg/dL...",dev_35645,aristotle,False,112,420,"(48, 49)",C0008976,Clinical Trials,C0086418,Homo sapiens,[Research Activity],[Eukaryote],"[[0, 1], [11, 13], [23, 24], [30, 33], [35, 36], [39, 40], [42, 44], [45, 47], [48, 49], [50, 51], [56, 57], [83, 85], [93, 94]]","[C1831808, C0004238, C0010294, C0206034, C1831808, C0038454, C0040038, C0004238, C0008976, C0008976, C1831808, C0428279, C1831808]","[C1831808, C0004238, C0010294, C0206034, C1831808, C0038454, C0019087, C0004238, C0086418, C0008976, C1831808, C1278055, C1831808]","[dev_35637, dev_35638, dev_35639, dev_35640, dev_35641, dev_35642, dev_35643, dev_35644, dev_35645, dev_35646, dev_35647, dev_35648, dev_35649]","[Intellectual Property, Homo sapiens, Research Activities, Scientific Name, COSTART, Inventions, Adreus, Not at all, Admiratio, Atropha, Description, Applaud, Alouatta, Sudafed, Anecdotes, Necator, AT 125, Empirical Research, Amen, Health Care Organization, research, entity - organization, Abulia, Heliamphora, Epoch, World Health Organization, Encyclopedias, Cardiovascular Agents, Company (organization), Geographic Locations]","[0.0575110205, 0.8884078264, 0.0175925

In [37]:
all_types = set()
for i, row in dev_preds_df.iterrows():
    for f in row["wiki_gld"]:
        all_types.add(f)
    
print(all_types)

{'Health Care Activity', 'Body System', 'Eukaryote', 'Intellectual Product', 'Food', 'Injury or Poisoning', 'Professional or Occupational Group', 'Spatial Concept', 'Clinical Attribute', 'Bacterium', 'Population Group', 'Virus', 'Chemical', 'Biologic Function', 'Finding', 'Anatomical Structure', 'Organization', 'Medical Device', 'Biomedical Occupation or Discipline', 'Body Substance', 'Research Activity'}


In [18]:
def compute_fuzz_score(df):
    crc = 0
    no_cands = 0
    indices = []
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        cand_names = row["cand_names"]
        if len(cand_names) == 0:
            no_cands += 1
            continue
        sp_l, sp_r = row["span"]
        al = " ".join(row["sentence"].split()[sp_l:sp_r])
        r = process.extractOne(al, cand_names)
    #     print(row["cands"], r)
        gld = row["gold_title"]
        if r[0] == gld:
            crc += 1
            indices.append(i)

    print(crc, no_cands, df.shape[0], crc/(df.shape[0]-no_cands))
    return indices

In [20]:
print("DEV")
indices = compute_fuzz_score(dev_df)
print("DEV WRONG")
indices = compute_fuzz_score(dev_preds_df[dev_preds_df["pred_qid"] != dev_preds_df["gold_qid"]])
print(dev_preds_df.shape)
print(dev_preds_df[dev_preds_df["wiki_gld"] != dev_preds_df["wiki_pred"]].shape)

(7555, 25)


In [21]:
print(sorted(list(errors_by_type(dev_preds_df, "wiki_gld").items()), key=lambda x:x[1], reverse=True))

[('Chemical', 3707), ('Biologic Function', 3040), ('Health Care Activity', 2096), ('Anatomical Structure', 1719), ('Finding', 1372), ('Intellectual Product', 910), ('Eukaryote', 907), ('Spatial Concept', 849), ('Research Activity', 535), ('Medical Device', 335), ('Population Group', 276), ('Bacterium', 257), ('Organization', 196), ('Injury or Poisoning', 181), ('Food', 149), ('Professional or Occupational Group', 133), ('Virus', 114), ('Clinical Attribute', 107), ('Biomedical Occupation or Discipline', 86), ('Body Substance', 70), ('Body System', 29)]


In [53]:
# DOCUMENT CONTEXT
def collect_doc_data(df):
    df = df.sort_values(by = ["doc_id", "sent_idx"])
    res = {}
    cur_res = {"first_sent": {"aliases": [], "gold_titles": [], "pred_titles": []}, 
               "all_doc": {"aliases": [], "gold_titles": [], "pred_titles": []}, 
               "rest_doc": {"aliases": [], "gold_titles": [], "pred_titles": []}}
    pred_doc_i = df.iloc[0]["doc_id"]
    first_sent_i = df.iloc[0]["sent_idx"]
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        doc_id = row["doc_id"]
        sent_idx = row["sent_idx"]
        if doc_id != pred_doc_i:
            res[doc_id] = cur_res
            first_sent_i = sent_idx
            pred_doc_i = doc_id
            cur_res = {"first_sent": {"aliases": [], "gold_titles": [], "pred_titles": []}, 
                       "all_doc": {"aliases": [], "gold_titles": [], "pred_titles": []}, 
                       "rest_doc": {"aliases": [], "gold_titles": [], "pred_titles": []}}
        if sent_idx == first_sent_i:
            cur_res["first_sent"]["aliases"].append(row["real_alias"])
            cur_res["first_sent"]["gold_titles"].append(row["gold_title"])
            cur_res["first_sent"]["pred_titles"].append(row["pred_title"])
        else:
            cur_res["rest_doc"]["aliases"].append(row["real_alias"])
            cur_res["rest_doc"]["gold_titles"].append(row["gold_title"])
            cur_res["rest_doc"]["pred_titles"].append(row["pred_title"])
        cur_res["all_doc"]["aliases"].append(row["real_alias"])
        cur_res["all_doc"]["gold_titles"].append(row["gold_title"])
        cur_res["all_doc"]["pred_titles"].append(row["pred_title"])
    return res
        

In [54]:
res = collect_doc_data(dev_preds_df)

100%|██████████| 40754/40754 [00:04<00:00, 8261.16it/s]


In [55]:
from collections import defaultdict, Counter

men_dct_doc = {}
men_dct_doc_pred = {}
for doc_i in res:
    men_dct_doc[doc_i] = defaultdict(Counter)
    men_dct_doc_pred[doc_i] = defaultdict(Counter)
    for al, gld, pred in zip(res[doc_i]["all_doc"]["aliases"], res[doc_i]["all_doc"]["gold_titles"], res[doc_i]["all_doc"]["pred_titles"]):
        men_dct_doc[doc_i][al][gld] += 1
        men_dct_doc_pred[doc_i][al][pred] += 1

In [56]:
num_m = 0
total_glds = 0
num_glds = []
max_glds = []
for doc_i in men_dct_doc:
    for men in men_dct_doc[doc_i]:
        num_m += 1
        num_glds.append(len(men_dct_doc[doc_i][men]))
        max_glds.append(max(men_dct_doc[doc_i][men].values()))
        total_glds += sum(men_dct_doc[doc_i][men].values())
            
print("Total Unique Mens", num_m, "Total Glds", total_glds, "90th Per Men", np.percentile(num_glds, 90), "90th Per Max", np.percentile(max_glds, 90), np.mean(max_glds))

Total Unique Mens 24241 Total Glds 40726 90th Per Men 1.0 90th Per Max 3.0 1.6729095334350894
Total Unique Mens 24241 Total Preds 40726 90th Per Men 1.0 90th Per Max 3.0 1.6729095334350894


In [57]:
# Number of times gold QID in first sentence is in doc
num_qids_match_gld_first = 0
num_qids_match_pred_first = 0
num_qids_match_gld_first_crc = 0
num_qids_match_pred_first_crc = 0
for doc_i in res:
    first_als = res[doc_i]["first_sent"]["aliases"]
    first_glds = res[doc_i]["first_sent"]["gold_titles"]
    first_preds = res[doc_i]["first_sent"]["pred_titles"]
    for al, gld, pred in zip(res[doc_i]["rest_doc"]["aliases"], res[doc_i]["rest_doc"]["gold_titles"], res[doc_i]["rest_doc"]["pred_titles"]):
        if gld in first_glds:
            num_qids_match_gld_first += 1
            if pred == gld:
                num_qids_match_gld_first_crc += 1
        if gld in first_preds:
            num_qids_match_pred_first += 1
            if pred == gld:
                num_qids_match_pred_first_crc += 1
print("GOLD: Total", num_qids_match_gld_first, "CRC", num_qids_match_gld_first_crc)
print("GOLD: Total", num_qids_match_pred_first, "CRC", num_qids_match_pred_first_crc)

GOLD: Total 12021 CRC 6423
GOLD: Total 7567 CRC 5852


In [23]:
# DESCRIPTION
import pickle

in_f = "/dfs/scratch0/lorr1/projects/bootleg-data/data/medmentions_0203/qid2def.pickle"
qid2def = pickle.load(open(in_f, "rb"))
print(len(qid2def))

c = 0
for i, row in tqdm(dev_preds_df.iterrows(), total=dev_preds_df.shape[0]):
    if row["pred_qid"] != row["gold_qid"]:
        if row["gold_qid"] in qid2def and len(qid2def[row["gold_qid"]]) > 5:
            c += 1

errs = dev_preds_df[dev_preds_df["pred_qid"] != dev_preds_df["gold_qid"]]
print(errs.shape[0], c, c/errs.shape[0])

  1%|          | 421/40754 [00:00<00:09, 4208.95it/s]

159195


100%|██████████| 40754/40754 [00:04<00:00, 9314.42it/s]

17318 11560 0.6675135696962697


In [120]:
# out_f = data_dir/"qid2titledesc.json"
# new_q2title_enhances = {}
# for q in q2title:
#     title = q2title[q]
#     desc=qid2def.get(q, "")
#     if len(desc) > 0:
#         desc = " [SEP] " + desc
#     new_title = f"{title}{desc}".strip()
#     new_q2title_enhances[q] = new_title
    
# json.dump(new_q2title_enhances, open(out_f, "w"))

# Robogym

In [76]:
dataset_med = Dataset.from_feather(dev_pred_file.parent / "dev_results.feather")
dataset_train = Dataset.from_pandas(train_df)
dataset_med.head(10)

,sentence,sent_idx,doc_idx,aliases,span,slices,alias,alias_idx,is_gold_label,gold_qid,pred_qid,gold_title,pred_title,all_gold_qids,all_pred_qids,gold_label_aliases,all_is_gold_labels,all_spans,qid2types_gld,qid2types_pred,num_cands,cand_names,cand_probs,in_cand,qid_cnt,pred_qid_cnt,index
0,Primary Management and Outcome - Open Laryngotracheal Trauma Acute external injury to the larynx is both life threatening and a potential long term management challenge .,34508,28384898,"[management, open laryngotracheal trauma, larynx, life threatening]","[1, 2]",[],management,0,True,C0376636,C0376636,"Disease Management [SEP] A broad approach to appropriate coordination of the entire disease treatment process that often involves shifting away from more expensive inpatient and acute care to areas such as preventive medicine, patient counseling and education, and outpatient care. This concept includes implications of appropriate versus inappropriate therapy on the overall cost and clinical outcome of a particular disease. (From Hosp Pharm 1995 Jul;30(7):596)","Disease Management [SEP] A broad approach to appropriate coordination of the entire disease treatment process that often involves shifting away from more expensive inpatient and acute care to areas such as preventive medicine, patient counseling and education, and outpatient care. This concept includes implications of appropriate versus inappropriate therapy on the overall cost and clinical outcome of a particular disease. (From Hosp Pharm 1995 Jul;30(7):596)","[C0376636, C0339881, C0023078, C2826244]","[C0376636, C1293340, C0023078, C2826244]","[management, open laryngotracheal trauma, larynx, life threatening]","[True, True, True, True]","[[1, 2], [5, 8], [13, 14], [16, 18]]",[Health Care Activity],[Health Care Activity],10,"[Disease Management [SEP] A broad approach to appropriate coordination of the entire disease treatment process that often involves shifting away from more expensive inpatient and acute care to areas such as preventive medicine, patient counseling and education, and outpatient care. This concept includes implications of appropriate versus inappropriate therapy on the overall cost and clinical outcome of a particular disease. (From Hosp Pharm 1995 Jul;30(7):596), Anger management case manageme...","[0.9968745112, 3.71208e-05, 0.0001725003, 5.78583e-05, 2.09597e-05, 0.0014233163, 0.000933026, 1.70495e-05, 8.3372e-05, 0.0003803039]",True,78,78,0
1,Primary Management and Outcome - Open Laryngotracheal Trauma Acute external injury to the larynx is both life threatening and a potential long term management challenge .,34508,28384898,"[management, open laryngotracheal trauma, larynx, life threatening]","[5, 8]",[],open laryngotracheal trauma,1,True,C0339881,C1293340,Injury of larynx [SEP] Trauma to the larynx.,Laryngotracheal resection,"[C0376636, C0339881, C0023078, C2826244]","[C0376636, C1293340, C0023078, C2826244]","[management, open laryngotracheal trauma, larynx, life threatening]","[True, True, True, True]","[[1, 2], [5, 8], [13, 14], [16, 18]]",[Injury or Poisoning],[Health Care Activity],10,"[Laryngotracheal, Laryngeal cleft [SEP] Presence of a gap in the posterior laryngotracheal wall with a continuity between the larynx and the esopahagus. [HPO:probinson, pmid:17178945], Laryngotracheal hemangioma, Laryngotracheal oedema, Laryngotracheal tube [SEP] A developmental feature of the early embryonic foregut on the ventral wall of the pharynx, generally forming during gestational week four, which is the primordium of the larynx, trachea, bronchi, and lungs., Laryngotracheal resectio...","[0.0035692549, 0.0943966657, 0.0491938218, 0.0264155995, 0.1099506542, 0.3033444881, 0.1413491964, 0.1965642124, 0.0637055114, 0.0115105379]",True,0,0,1
2,Primary Management and Outcome - Open Laryngotracheal Trauma Acute external injury to the larynx is both life threatening and a potential long term management challenge .,34508,28384898,"[management, open laryngotracheal trauma, 

In [87]:
# Some preprocessing of training data
top_100_qids = [p[0] for p in sorted(list(qid2cnt.items()), key=lambda x: x[1], reverse=True)][:100]
print(top_100_qids)
all_mentions = set(dataset_train["alias"])
men_qid_pairs = Counter(zip(dataset_train['alias'], dataset_train["gold_qid"]))

['C2603343', 'C0087111', 'C0936012', 'C1171362', 'C0220825', 'C0043210', 'C0007634', 'C0086418', 'C0011900', 'C1257890', 'C0017262', 'C0017337', 'C0679646', 'C0543467', 'C0012634', 'C0005516', 'C0025929', 'C1705920', 'C0027651', 'C1522577', 'C0150312', 'C0033684', 'C0040300', 'C1457887', 'C0027361', 'C0025663', 'C0184511', 'C0237401', 'C0184661', 'C0442726', 'C0035168', 'C0006104', 'C1515655', 'C0025266', 'C3161035', 'C1446409', 'C0678594', 'C3714514', 'C0021368', 'C0019994', 'C0035648', 'C0009566', 'C0162638', 'C0006826', 'C0027882', 'C0599755', 'C0681814', 'C0013227', 'C0229671', 'C0011860', 'C0014406', 'C0024485', 'C0032098', 'C0205148', 'C0041904', 'C0001792', 'C0221198', 'C0028754', 'C0080105', 'C0032105', 'C0449774', 'C0684224', 'C0011847', 'C0032961', 'C0017428', 'C0332293', 'C0020538', 'C0041296', 'C1101610', 'C1327622', 'C0014442', 'C0026882', 'C0003062', 'C0041703', 'C0920317', 'C0205160', 'C0008972', 'C1305855', 'C0012155', 'C0038454', 'C1511790', 'C0700287', 'C0456389', 'C0

In [78]:
# RG Helpers
import numpy as np

def accuracy(true: np.array, pred: np.array):
    """
    Your function for computing accuracy.    
    """
    return np.mean([t == p for t, p in zip(true, pred)])

def top_w(true: list, cand_probs: list, cand_names: list, threshold: float = 0.3):
    res = []
    for i, cand_p in enumerate(cand_probs):
        cand_p = np.array(cand_p)
        cand_n = np.array(cand_names[i])
        predicted_qs = [title2q[p] for p in cand_n[cand_p > threshold]]
        if any(true[i] in p for p in predicted_qs):
            res.append(1)
        else:
            res.append(0)
    return np.mean(np.array(res))

def print_metrics(slices, m_func="accuracy", suffix=""):
    metrics = {}
    for sl in slices:
        if m_func == "accuracy":
            metrics[sl.identifier] = str(accuracy(true=sl["gold_qid"], pred=sl[f'pred_qid{suffix}'])) + " (" + str(len(sl)) + ")"
        elif m_func == "top2weight":
            metrics[sl.identifier] = str(top_w(sl["gold_qid"], sl[f"cand_probs{suffix}"], sl[f"cand_names{suffix}"], 0.3)) + " (" + str(len(sl)) + ")"
        else:
            print(f"You don't have a metric for this")
            return None
    print(json.dumps(metrics, indent=4))

In [89]:
sp_multi_word_mention = rg.ScoreSubpopulation(
    intervals=[(0,0), (1,1)],
    identifiers=[Identifier('SingleWordSpan'), Identifier('MultiWordSpan')],
    score_fn=lambda batch, columns: [len(e.split())>1 for e in batch[columns[0]]],
)

sp_mention_len = rg.ScoreSubpopulation(
    intervals=[('0%', '5%'), ('95%', '100%')],
    identifiers=[Identifier('MentionLenLow'), Identifier('MentionLenHigh')],
    score_fn=lambda batch, columns: [len(e.split()) for e in batch[columns[0]]],
)

sp_mention_unseen = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 1)],
    identifiers=[Identifier('MenUnseen'), Identifier('MenNotUnseen')],
    score_fn=lambda batch, columns: [e in all_mentions for e in batch[columns[0]]],
)

sp_mention_direct_match = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 1)],
    identifiers=[Identifier('MenNotMatchTitle'), Identifier('MenMatchTitle')],
    score_fn=lambda batch, columns: [int(e.lower()==f.lower()) for e, f in zip(batch[columns[0]], batch[columns[1]])],
)

sp_is_qid_unseen = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 1)],
    identifiers=[Identifier('QIDUnseen'), Identifier('QIDNotUnseen')],
    score_fn=lambda batch, columns: [int(e > 0) for e in batch[columns[0]]],
)

sp_qid_cnt = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 5), (6, 10), (11, 50), (51, 100), (101, 500), (501, 500000000)],
    identifiers=[Identifier('0'), Identifier('1-5'), Identifier('6-10'), Identifier('11-50'), Identifier('51-100'), Identifier('101-500'), Identifier('501-')],
    score_fn=lambda batch, columns: [e for e in batch[columns[0]]],
)

sp_qid_top100 = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 1)],
    identifiers=[Identifier('QIDNotTop100'), Identifier('QIDTop100')],
    score_fn=lambda batch, columns: [int(e in top_100_qids) for e in batch[columns[0]]],
)

sp_mention_len = rg.ScoreSubpopulation(
    intervals=[('0%', '5%'), ('95%', '100%')],
    identifiers=[Identifier('MentionLenLow'), Identifier('MentionLenHigh')],
    score_fn=lambda batch, columns: [len(e.split()) for e in batch[columns[0]]],
)

sp_mention_qid_pop = rg.ScoreSubpopulation(
    intervals=[('0%', '5%'), ('90%', '100%')],
    identifiers=[Identifier('Unpopular'), Identifier('Popular')],
    score_fn=lambda batch, columns: [men_qid_pairs[(e,f)] for e, f in zip(batch[columns[0]], batch[columns[1]])],
)

sp_sent_len = rg.ScoreSubpopulation(
    intervals=[('0%', '5%'), ('95%', '100%')],
    identifiers=[Identifier('LenSentence-Low'), Identifier('LenSentence-High')],
    score_fn=lambda batch, columns: [len(e) for e in batch[columns[0]]],
)

sp_type_overlap = rg.ScoreSubpopulation(
    intervals=[(0, 0), (1, 1), (2, 2), (3, 3)],
    identifiers=[Identifier('0'), Identifier('1'), Identifier('2'), Identifier('3')],
    score_fn=lambda batch, columns: [len(set(e).intersection(f)) for e, f in zip(batch[columns[0]], batch[columns[1]])],
)

In [90]:
print(f"All Spans")
print(accuracy(dataset_med["pred_qid"], dataset_med["gold_qid"]))

print("sp_multi_word_mention")
sls, mat = sp_multi_word_mention(dataset_med, ['alias'])
print_metrics(sls)

print("sp_mention_len")
sls, mat = sp_mention_len(dataset_med, ["alias"])
print_metrics(sls)

print("sp_mention_unseen")
sls, mat = sp_mention_unseen(dataset_med, ["alias"])
print_metrics(sls)

print("sp_mention_direct_match")
sls, mat = sp_mention_direct_match(dataset_med, ["alias", "gold_title"])
print_metrics(sls)

print("sp_is_qid_unseen")
sls, mat = sp_is_qid_unseen(dataset_med, ["qid_cnt"])
print_metrics(sls)

print("sp_qid_cnt")
sls, mat = sp_qid_cnt(dataset_med, ["qid_cnt"])
print_metrics(sls)

print("sp_qid_top100")
sls, mat = sp_qid_top100(dataset_med, ["gold_qid"])
print_metrics(sls)

print("sp_mention_len")
sls, mat = sp_mention_len(dataset_med, ["alias"])
print_metrics(sls)

print("sp_mention_qid_pop")
sls, mat = sp_mention_qid_pop(dataset_med, ["alias", "gold_qid"])
print_metrics(sls)

print("sp_sent_len")
sls, mat = sp_sent_len(dataset_med, ["sentence"])
print_metrics(sls)

print("sp_type_overlap")
sls, mat = sp_type_overlap(dataset_med, ["qid2types_gld", "qid2types_pred"])
print_metrics(sls)

All Spans
0.5750601167983511
sp_multi_word_mention
{
    "Feather(path=\/dfs\/scratch1\/lorr1\/projects\/bootleg\/logs_medmentions_0203\/cls_type_desc_512\/2021_02_17\/16_26_27\/e5908368\/dev\/checkpoint_4.0\/dev_results.feather) -> SingleWordSpan @ alias": "0.6480461027215749 (23773)",
    "Feather(path=\/dfs\/scratch1\/lorr1\/projects\/bootleg\/logs_medmentions_0203\/cls_type_desc_512\/2021_02_17\/16_26_27\/e5908368\/dev\/checkpoint_4.0\/dev_results.feather) -> MultiWordSpan @ alias": "0.4728814557446558 (16981)"
}
sp_mention_len
{
    "Feather(path=\/dfs\/scratch1\/lorr1\/projects\/bootleg\/logs_medmentions_0203\/cls_type_desc_512\/2021_02_17\/16_26_27\/e5908368\/dev\/checkpoint_4.0\/dev_results.feather) -> MentionLenLow @ alias": "0.6480461027215749 (23773)",
    "Feather(path=\/dfs\/scratch1\/lorr1\/projects\/bootleg\/logs_medmentions_0203\/cls_type_desc_512\/2021_02_17\/16_26_27\/e5908368\/dev\/checkpoint_4.0\/dev_results.feather) -> MentionLenHigh @ alias": "0.39508310249307477 

# JUNK

In [28]:
typemap = {'T059': 'T058', 'T060': 'T058', 'T061': 'T058', 'T063': 'T062', 'T039': 'T038', 'T040': 'T038', \
          'T041': 'T038', 'T042': 'T038', 'T043': 'T038', 'T044': 'T038', 'T045': 'T038', 'T046': 'T038', 'T047': 'T038', \
          'T048': 'T038', 'T191': 'T038', 'T049': 'T038', 'T050': 'T038', 'T008':'T204', 'T010':'T204', \
          'T011':'T204', 'T012':'T204', 'T013':'T204', 'T014':'T204', 'T015':'T204', 'T016':'T204', 'T018': 'T017', \
          'T190': 'T017','T019': 'T017','T020': 'T017', 'T021': 'T017', 'T023':'T017', 'T024':'T017', 'T025':'T017', \
          'T026':'T017', 'T028':'T017', 'T203': 'T074', 'T120':'T103', 'T121':'T103', 'T195':'T103', 'T122':'T103', \
          'T123':'T103', 'T125':'T103', 'T126':'T103', 'T127':'T103', 'T129':'T103', 'T192':'T103', 'T130':'T103', \
          'T131':'T103', 'T104':'T103', 'T109':'T103', 'T114':'T103', 'T116':'T103', 'T197':'T103', 'T196':'T103', \
          'T034': 'T033', 'T184':'T033', 'T030':'T082', 'T029':'T082', 'T085':'T082', 'T086':'T082', 'T087':'T082', \
           'T088':'T082', 'T083':'T082', 'T185':'T170', 'T089':'T170', 'T095':'T092', 'T093':'T092', 'T094':'T092', \
          'T002': 'T204', 'T004':'T204'}

semantic_type = [l.strip().split('|') for l in open("/dfs/scratch0/lorr1/projects/bootleg-data/data/medmentions_0203/SemanticTypes_2018AB.txt")]
semantic_type = {p[1]:p[2] for p in semantic_type}
new_typemap = {semantic_type[k]: semantic_type[v] for k,v in typemap.items()}
print(json.dumps(new_typemap, indent=4))

{
    "Laboratory Procedure":"Health Care Activity",
    "Diagnostic Procedure":"Health Care Activity",
    "Therapeutic or Preventive Procedure":"Health Care Activity",
    "Molecular Biology Research Technique":"Research Activity",
    "Physiologic Function":"Biologic Function",
    "Organism Function":"Biologic Function",
    "Mental Process":"Biologic Function",
    "Organ or Tissue Function":"Biologic Function",
    "Cell Function":"Biologic Function",
    "Molecular Function":"Biologic Function",
    "Genetic Function":"Biologic Function",
    "Pathologic Function":"Biologic Function",
    "Disease or Syndrome":"Biologic Function",
    "Mental or Behavioral Dysfunction":"Biologic Function",
    "Neoplastic Process":"Biologic Function",
    "Cell or Molecular Dysfunction":"Biologic Function",
    "Experimental Model of Disease":"Biologic Function",
    "Animal":"Eukaryote",
    "Vertebrate":"Eukaryote",
    "Amphibian":"Eukaryote",
    "Bird":"Eukaryote",
    "Fish":"Eukaryote",
 